# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

Recall Problem 1 from hw3t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.nodes[state]['demand'] = 1
G.nodes['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edges[s1, s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [4]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    
    #DEMAND LISTS TO GET SOURCE AND SINK VALS
    positive_demands = []
    
    negative_demands = []
    
    g = graph.copy()
    
    #APPEND DEMANDS TO LISTS
    for node in g.nodes():
        if g.nodes[node]['demand'] > 0:
            positive_demands.append(g.nodes[node]['demand'])
        elif g.nodes[node]['demand'] < 0:
            negative_demands.append(g.nodes[node]['demand'])
    
    #SUM Ds TO CHECK THAT GRAPH IS FEASIBLE & GET SUPER SOURCE/SINK CAPCITIES
    total_positive = sum(positive_demands)
    total_negative = sum(negative_demands)
    if total_positive != -1*total_negative:
        return 'NetworkXUnfeasible'

    H = graph.copy()
    
    H.add_node('super_sink')
    H.nodes['super_sink']['demand'] = total_positive
    
    H.add_node('super_source')
    H.nodes['super_source']['demand'] = total_negative
    
    #CONNECT SINKS TO SUPER SINK
    for s1 in H.nodes():
        for s2 in H.nodes():
            if s2 == 'super_sink' and s1 != 'super_sink' and H.nodes[s1]['demand'] > 0:
                H.add_edge(s1, s2)
                
    #CONNECT SUPER SOURCE TO SOURCES
    for s1 in H.nodes():
        for s2 in H.nodes():
            if s1 == 'super_source' and s2 != 'super_source' and H.nodes[s1]['demand'] < 0:
                H.add_edge(s1, s2)
            
    #ADDING NEW EDGE CAPCITIES
    for (s1, s2) in H.edges():
        if s2 == 'super_sink' and s1 != 'super_sink':
            H.edges[s1, s2]['capacity'] = H.nodes[s1]['demand']
        elif s1 == 'super_source' and s2 != 'super_source':
            H.edges[s1, s2]['capacity'] = -1*H.nodes[s2]['demand']
    
    #GET MAX FLOW LARRY
    max_flow = nx.maximum_flow(H, 'super_source', 'super_sink')
    
    #GET RID OF FLOWS TO SUPER SINK
    for (node, connected) in max_flow[1].items():
        connected.pop('super_sink', None)
    
    #GET RID OF SUPER SINK AND SUPER SOURCE ENTRIES
    max_flow[1].pop('super_sink', None)
    max_flow[1].pop('super_source', None)
        
    return max_flow[1]

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [5]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    mf_copy = flow.copy()
    mf_copy

    #INITIALIZE VALUES FLOW VALUES TO 0
    for state in mf_copy:
        mf_copy[state]['flow_out'] = 0
        mf_copy[state]['flow_in'] = 0


    #NESTED FOR LOOP FOR STATE FLOW INTERACTIONS - FLOW IN AND OUT ADDED UP
    for state in mf_copy:
        borders = mf_copy[state]
        for border in borders:
            if border not in ('flow_in', 'flow_out'):
                flow_considered = mf_copy[state][border]
                mf_copy[state]['flow_out'] += flow_considered
                mf_copy[border]['flow_in'] += flow_considered

    #ANNNNNNND NET FLOW
    for state in mf_copy:
        mf_copy[state]['net_flow'] = mf_copy[state]['flow_in'] - mf_copy[state]['flow_out']

    return_copy = {}

    for state in mf_copy:
        return_copy[state] = mf_copy[state]['net_flow']

    return return_copy

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [6]:
flow = flow_with_demands(G)
div = divergence(flow)
print ("Flow satisfies all demands:", all(div[n] == G.nodes[n]['demand'] for n in G.nodes()))

Flow satisfies all demands: True
